In [1]:
import foolbox
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [26]:
transform = transforms.Compose([
    transforms.ToTensor()
])

test = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(
    test, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified


In [35]:
def generate_adversarials(attack, dataloader, save_name=None):
    
    adversarials = []
    for x, y in dataloader:
        advs = attack(x.numpy(), y.numpy())
        adversarials.extend(advs)
        
    adversarials = np.array(adversarials)
    
    if save_name:
        np.save(f"adversarials/{save_name}", adversarials)
        
    return adversarials

### Loading model

In [23]:
from vgg import *

VGG_16 = [64, 64, 'pooling', 128, 128, 'pooling', 256, 256,
          256, 'pooling', 512, 512, 512, 'pooling', 512, 512, 512, 'pooling']

net = VGG(VGG_16)
net = net.to(device)
net.eval()

net.load_state_dict(torch.load('models_weights/vggModel'))

<All keys matched successfully>

In [24]:
preprocessing = dict(axis=-3)
fmodel = foolbox.models.PyTorchModel(net, bounds=(0, 1), num_classes=10, preprocessing=preprocessing)

In [25]:
fgsm = foolbox.attacks.FGSM(fmodel)
ddna06 = foolbox.attacks.DecoupledDirectionNormL2Attack(fmodel, criterion=foolbox.criteria.TargetClassProbability(6, p=.6))
ddna07 = foolbox.attacks.DecoupledDirectionNormL2Attack(fmodel, criterion=foolbox.criteria.TargetClassProbability(6, p=.7))
ddna075 = foolbox.attacks.DecoupledDirectionNormL2Attack(fmodel, criterion=foolbox.criteria.TargetClassProbability(6, p=.75))

In [8]:
generate_adversarials(fgsm, test_loader, 'fgsm')
generate_adversarials(fgsm, test_loader, 'ddna06')
generate_adversarials(fgsm, test_loader, 'ddna07')
generate_adversarials(fgsm, test_loader, 'ddna075')

C:\ProgramData\Anaconda3\lib\site-packages\foolbox\attacks\base.py:92: UserWarning: GradientSignAttack did not find an adversarial, maybe the model or the criterion is not supported by this attack.
  warnings.warn(


foolbox 2.4.0 documentation
https://foolbox.readthedocs.io/en/v2.4.0/user/examples.html